In [1]:
import pandas as pd
from engine.hyperparameters_tune_raytune import deepscreen_hyperparameter_tuneing
from utils.configurations import configs
import os

/home/sjinich/disco/che_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-02-29 13:22:13,721	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-29 13:22:13,883	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2024-02-29 13:22:15,651	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.


In [2]:
import warnings
warnings.filterwarnings("ignore", ".*Consider increasing the value of the `num_workers` argumen*")

data = pd.read_csv("/home/sjinich/disco/TrypanoDEEPscreen/.data/processed/CHEMBL2581.csv")
    
search_space_deepscreen = configs.get_hyperparameters_search()

tuner = deepscreen_hyperparameter_tuneing(
    data=data,
    data_split_mode="non_random_split",
    search_space=search_space_deepscreen,
    target="chembl2581_trial",
    experiments_result_path="../../.experiments",
    **configs.get_hyperparameters_search_setup()
    )

result = tuner.tune_deepscreen()


(TorchTrainer pid=1838392) Started distributed worker processes: 
(TorchTrainer pid=1838392) - (ip=10.1.103.91, pid=1838584) world_rank=0, local_rank=0, node_rank=0
(RayTrainWorker pid=1838584) Setting up process group for: env:// [rank=0, world_size=1]
(RayTrainWorker pid=1838584) INFO: Using hyperparameters [('fully_layer_1', 256), ('fully_layer_2', 32), ('drop_rate', 0.5), ('learning_rate', 0.0001), ('batch_size', 32), ('experiment_result_path', '/big/lab/sjinich/TrypanoDEEPscreen/.experiments/chembl2581_trial'), ('temperature_scaleing', 1)]
(RayTrainWorker pid=1838584) GPU available: False, used: False
(RayTrainWorker pid=1838584) TPU available: False, using: 0 TPU cores
(RayTrainWorker pid=1838584) IPU available: False, using: 0 IPUs
(RayTrainWorker pid=1838584) HPU available: False, using: 0 HPUs
(RayTrainWorker pid=1838584) [rank: 0] Seed set to 123
(RayTrainWorker pid=1838584) Missing logger folder: /home/sjinich/ray_results/TorchTrainer_2024-02-29_13-22-15/TorchTrainer_90bca_0

In [38]:
bestmet = result.get_best_result(metric="val_mcc",mode="max").metrics
{i:j for i,j in bestmet.items() if i.find("val") != -1 }

{'val_loss': 0.6341113448143005,
 'val_acc': 0.6181318759918213,
 'val_auroc': 0.7709251046180725,
 'val_auroc_15': 0.5830983519554138,
 'val_calibration_error': 0.16824299097061157,
 'val_f1': 0.627345860004425,
 'val_mcc': 0.30024105310440063,
 'val_prec': 0.801369845867157,
 'val_recall': 0.515418529510498}

In [3]:
ckpt_dir = os.path.join(result.get_best_result(metric="val_mcc",mode="max").checkpoint.path,"checkpoint.ckpt")

In [12]:
from engine.system import DEEPScreenClassifier
from datasets.datamodule import DEEPscreenDataModule

model = DEEPScreenClassifier.load_from_checkpoint(ckpt_dir,experiment_result_path="../../.experiments/chembl2581")

INFO: Using hyperparameters [('fully_layer_1', 256), ('fully_layer_2', 32), ('drop_rate', 0.5), ('learning_rate', 0.0001), ('batch_size', 32), ('experiment_result_path', '../../.experiments/chembl2581'), ('temperature_scaleing', 1)]


In [35]:
model.CHECKPOINT_HYPER_PARAMS_TYPE

'hparams_type'

In [5]:
data = pd.read_csv("../../.data/processed/CHEMBL2581.csv")
datamodule = DEEPscreenDataModule(data=data,target_id="CHEMBL2581",batch_size=32,experiment_result_path="../../.experiments/chembl2581",data_split_mode="non_random_split",tmp_imgs=True)


In [6]:
from lightning import Trainer

trainer = Trainer()

trainer.test(model,datamodule=datamodule)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
INFO: Using a total of 2270 datapoints
INFO: non_random_split datasets splited train=1452,validation=364,test=454


Testing DataLoader 0: 100%|██████████| 15/15 [00:04<00:00,  3.50it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_acc            0.6453744769096375
       test_auroc           0.7404670119285583
      test_auroc_15         0.5651069283485413
 test_calibration_error     0.15835818648338318
         test_f1            0.6298850774765015
        test_loss           0.6269093751907349
        test_mcc            0.32578131556510925
        test_prec           0.7611111402511597
       test_recall          0.5372549295425415
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


/home/sjinich/disco/che_env/lib/python3.8/site-packages/lightning/pytorch/trainer/connectors/logger_connector/result.py:211: You called `self.log('test_mcc', ...)` in your `test_step` but the value needs to be floating point. Converting it to torch.float32.


[{'test_loss': 0.6269093751907349,
  'test_acc': 0.6453744769096375,
  'test_auroc': 0.7404670119285583,
  'test_auroc_15': 0.5651069283485413,
  'test_calibration_error': 0.15835818648338318,
  'test_f1': 0.6298850774765015,
  'test_mcc': 0.32578131556510925,
  'test_prec': 0.7611111402511597,
  'test_recall': 0.5372549295425415}]

In [7]:
result_df = result.get_dataframe()
result_df.loc[2]

KeyError: 2

In [ ]:
dir(result.get_best_result(metric="val_mcc",mode="max").checkpoint)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__fspath__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slotnames__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_annotated',
 '_get_temporary_checkpoint_dir',
 '_uuid',
 'as_directory',
 'filesystem',
 'from_directory',
 'get_metadata',
 'path',
 'set_metadata',
 'to_directory',
 'update_metadata']

In [ ]:
result.get_best_result(metric="val_mcc",mode="max").config["train_loop_config"]["batch_size"]

{'fully_layer_1': 256,
 'fully_layer_2': 32,
 'learning_rate': 0.0001,
 'batch_size': 32,
 'drop_rate': 0.3}